# Imports

In [ ]:
import numpy as np

from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector, state_fidelity

from LogicalQ.Logical import LogicalCircuit, LogicalStatevector, logical_state_fidelity
from LogicalQ.Library.QECCs import steane_code
from LogicalQ.Library.HardwareModels import hardware_models_Quantinuum
from LogicalQ.Execution import execute_circuits

# Two Plaquette SU(2) Hamiltonian Model

In [ ]:
def TwoPlaquetteSU2TrotterStep(qc, x, t):
    qc.cx(1, 0)
        
    qc.ry(-1/2 * x * t, 0)
    qc.rz(-3/8 * t, 0)
    
    qc.cx(1, 0)
    
    qc.ry(-3/2 * x * t, 0)
    qc.rz(-9/8 * t, 1)
    
    qc.rz(-9/4 * t, 0)
    qc.ry(-3 * x * t, 1)

    qc.cx(0, 1)

    qc.ry(-1 * x * t, 1)
    
    qc.cx(0, 1)

    qc.ry(-3/2 * x * t, 0)
    qc.rz(-9/8 * t, 1)

    qc.cx(1, 0)

    qc.rz(-3/8 * t, 0)
    qc.ry(-1/2 * x * t, 0)

    qc.cx(1, 0)

def TwoPlaquetteSU2(x, dt, Nt, measure=True, mitigation=False):
    qc = QuantumCircuit(2)
    
    # Physical pass
    for _ in range(Nt):
        TwoPlaquetteSU2TrotterStep(qc, x, dt)

    if measure:
        qc.measure_all()

    # Mitigation passes
    if mitigation:
        # Forward-time half-pass
        for _ in range(Nt//2):
            TwoPlaquetteSU2TrotterStep(qc, x, dt)
        # Backward-time half-pass
        for _ in range(Nt//2):
            TwoPlaquetteSU2TrotterStep(qc, x, -dt)

        if measure:
            qc.measure_all()

    return qc

In [ ]:
qc = TwoPlaquetteSU2(2, 0.1, 1, measure=True, mitigation=False)
qc.draw("mpl")

# Exact Simulation

In [ ]:
qc_no_meas = TwoPlaquetteSU2(2, 0.1, 1, measure=False, mitigation=False)

sv_exact = Statevector(qc_no_meas)
sv_exact.draw("latex")

# Physical QuantumCircuit Simulation

In [ ]:
result_physical = execute_circuits(
    qc,
    backend="aer_simulator", method="statevector",
    hardware_model=hardware_models_Quantinuum["H2-2"], coupling_map=None,
    shots=int(1E6)
)[0]
result_physical

In [ ]:
counts = result_physical.get_counts()
total_counts = sum(list(counts.values()))
counts, total_counts

In [ ]:
amplitudes = []
for o in range(2**2):
    outcome_bin = bin(o)[2:].zfill(2)
    
    amplitudes.append((counts.get(outcome_bin, 0.0)/total_counts)**0.5)

sv = Statevector(amplitudes)
sv.draw("latex")

In [ ]:
fidelity_physical = state_fidelity(sv_exact, sv)
fidelity_physical

# LogicalCircuit Simulation

In [ ]:
lqc = LogicalCircuit.from_physical_circuit(qc, **steane_code)
lqc.draw("mpl")

In [ ]:
result_logical = execute_circuits(
    lqc,
    backend="aer_simulator", method="statevector",
    hardware_model=hardware_models_Quantinuum["H2-2"], coupling_map=None,
    shots=int(1E3)
)[0]
result_logical

In [ ]:
counts = result_logical.get_counts()
counts

In [ ]:
logical_counts = lqc.get_logical_counts(counts)
logical_counts

In [ ]:
lsv = LogicalStatevector.from_counts(counts, lqc.n_logical_qubits, **steane_code)
lsv.draw("latex")

In [ ]:
fidelity_logical = logical_state_fidelity(sv_exact, lsv)
fidelity_logical

# Quantinuum Hardware Execution

In [ ]:
result_hardware = execute_circuits(
    lqc_su2,
    backend="quantinuum_H2-2E",
    shots=int(1E3)
)
result_hardware